# Data Download

The purpose of this notebook is to download data from The National Map using the TNM Rest API. API documentation is [available here](https://tnmaccess.nationalmap.gov/api/v1/docs). A more comprehensive dataset documentation is [available here](../../docs/markdown/datasets.md).

Data downloaded:
1. **Watershed Boundary (WBD)**
2. **Digital Elevation Model (DEM)**
3. **National Land Cover Data (Land Cover, Land Cover Confidence, Fraction Impervious Surface, Fraction Impervious Descriptor)**
4. **Precipitation Data**


In [2]:
# Import necessary modules
import requests
import pandas as pd
from pathlib import Path
import json
import sys
import os
import warnings
import re
from datetime import datetime
from IPython.display import display, Markdown


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/alan/Data Science Projects/Unit-Hydrograph-Model/venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/alan/Data Science Projects/Unit-Hydrograph-Model/venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/alan/Data Science Projects/Unit-Hydrograph-Model/venv/lib

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/alan/Data Science Projects/Unit-Hydrograph-Model/venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/alan/Data Science Projects/Unit-Hydrograph-Model/venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/alan/Data Science Projects/Unit-Hydrograph-Model/venv/lib

AttributeError: _ARRAY_API not found

In [10]:
# Base path
project_base_path = Path.cwd().parent.parent

# Supress warnings
warnings.filterwarnings("ignore")

In [11]:
# Add 'src' to system path
sys.path.append(str(project_base_path / 'src'))

# Import modules
from dataDownload.download import download_shp, download_GeoTIFF, download_large_file

## 1. Watershed Boundary Dataset (WBD)

WBD is available thorough Rest API. (Refer to [dataset documentation here](../../docs/markdown/datasets.md) for more details). 
The HU-4 digits Watershed Boundary have the proper resolution. From the HU-2 region we select the HU-4 subregion. Below, are the available datasets.

### 1.1. Watershed Boundary Dataset availability

Below are the available WBD for download.

In [22]:
# Load the bounding box of the neighboring New York state
neighboring_ny_state_bbox_path = project_base_path / 'data' / 'raw' / 'geo' /'json' / 'ny_neighboring_bbox.json'
with open(neighboring_ny_state_bbox_path, 'r') as f:
    neighboring_ny_state_bbox_dict = json.load(f)

# Build the box as a string for feeding the request in the parameters
corners = ['bottom_left', 'bottom_right', 'top_right', 'top_left']
pairs = [f"{neighboring_ny_state_bbox_dict[corner][0]} {neighboring_ny_state_bbox_dict[corner][1]}" for corner in corners]
bbox = ",".join(pairs)

# Define the base URL for the TNM API
base_url = "https://tnmaccess.nationalmap.gov/api/v1/"

# Define parameters for the API request to query available datasets
params = {
    "polygon": bbox,  # Specify the area to search for
    "datasets": "National Watershed Boundary Dataset (WBD)",  # Specify Watershed Boundary Dataset
    "outputFormat": "JSON"  # Specify JSON output
}

# Send a GET request to the API
response = requests.get(base_url + "products", params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    response_json = response.json()
    
    # Display the dataset information
    print("Available Watershed Boundary Datasets:\n")
    for dataset in response_json.get("items", []):
        print(f"- Name: {dataset['title']}")
        print(f"  Extent: {dataset['extent']}")
        print(f"  Description: {dataset['body']}")
        print(f"  Metadata URL: {dataset['metaUrl']}\n")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")

Available Watershed Boundary Datasets:

- Name: USGS Watershed Boundary Dataset (WBD) - National (published 20250107) FileGDB
  Extent: National
  Description: The Watershed Boundary Dataset (WBD) is a comprehensive aggregated collection of hydrologic unit data consistent with the national criteria for delineation and resolution. It defines the areal extent of surface water drainage to a point except in coastal or lake front areas where there could be multiple outlets as stated by the "Federal Standards and Procedures for the National Watershed Boundary Dataset (WBD)" "Standard" (https://pubs.usgs.gov/tm/11/a3/). Watershed boundaries are determined solely upon science-based hydrologic principles, not favoring any administrative boundaries or special projects, nor particular program or agency. This dataset represents the hydrologic unit boundaries to the 12-digit (6th level) for the entire United States. Some areas may also include additional subdivisions representing the 14- and 16-dig

The response contains info about the available data set, including the download file. Below is an example of metadata for each Watershed Boundary Dataset.

In [23]:
", ".join([item for item in response_json.get("items", [])[0]])

'title, moreInfo, sourceId, sourceName, sourceOriginId, sourceOriginName, metaUrl, vendorMetaUrl, publicationDate, lastUpdated, dateCreated, sizeInBytes, extent, format, downloadURL, downloadURLRaster, previewGraphicURL, downloadLazURL, urls, datasets, boundingBox, bestFitIndex, body, processingUrl, modificationInfo'

In [24]:
# Filter files within the extent HU-2 digit in shapefile format
filtered_response = [
    record for record in response_json.get('items', [])
    if record.get('format') == 'Shapefile' and record.get('extent') == 'HU-2 Region'
]

# Save the filtered response to a file
export_path = project_base_path / 'data' / 'raw'/ 'json_docs' / 'watershed_boundary_dataset.json'
if not export_path.exists():
    try:
        with open(export_path, 'w') as f:
            json.dump(filtered_response, f, indent=4)
        print(f'\nWatershed Boundary Dataset exported successfully to {export_path}.\n')
    except Exception as err:
        print(f'Could not save the file. An error was encountered: {err}')
else:
    print(f'\nWatershed Boundary Dataset already exists at {export_path}.\n')
    
# Print the file download as markdown
display(Markdown(f'**{len(filtered_response)} files were found:**\n  '))
for record in filtered_response:
    print(f"- Name: {record['title']}")
    print(f"  Extent: {record['extent']}")
    print(f"  Description: {record['body']}")
    print(f"  Metadata URL: {record['metaUrl']}\n")


Watershed Boundary Dataset already exists at /Users/alan/Data Science Projects/Unit-Hydrograph-Model/data/raw/json_docs/watershed_boundary_dataset.json.



**4 files were found:**
  

- Name: USGS Watershed Boundary Dataset (WBD) for 2-digit Hydrologic Unit - 01 (published 20250108) Shapefile
  Extent: HU-2 Region
  Description: The Watershed Boundary Dataset (WBD) is a comprehensive aggregated collection of hydrologic unit data consistent with the national criteria for delineation and resolution. It defines the areal extent of surface water drainage to a point except in coastal or lake front areas where there could be multiple outlets as stated by the "Federal Standards and Procedures for the National Watershed Boundary Dataset (WBD)" "Standard" (https://pubs.usgs.gov/tm/11/a3/). Watershed boundaries are determined solely upon science-based hydrologic principles, not favoring any administrative boundaries or special projects, nor particular program or agency. This dataset represents the hydrologic unit boundaries to the 12-digit (6th level) for the entire United States. Some areas may also include additional subdivisions representing the 14- and 16-digit hydrologic

### 1.2. Watershed Boundary Dataset Download

In [25]:
# Download the Watershed Boundary shapefiles
for item in filtered_response:
    try:
        url = item.get('downloadURL')
        
        item_base_name = os.path.basename(url)
        item_local_path = project_base_path / 'data' / 'raw' / 'json_docs' / item_base_name
        
        if not item_local_path.exists():
            print(f'Downloading {item.get("title")}...')
            download_shp(url=url, filename=item_local_path, unzip=True)
            print(f'{item_base_name} downloaded and unzipped successfully.\n')
        else:
            print(f'{item_base_name} already exists.\n')

    except Exception as err:
        print(f'Failed to download or unzip file {item_base_name}: {err}')
        continue

Downloaded: WBD_01_HU2_Shape.zip
Extracted files to: /Users/alan/Data Science Projects/Unit-Hydrograph-Model/data/raw/json_docs/WBD_01_HU2_Shape
Deleted ZIP file: /Users/alan/Data Science Projects/Unit-Hydrograph-Model/data/raw/json_docs/WBD_01_HU2_Shape.zip
Function 'download_shp' executed in 30.0259 seconds.
WBD_01_HU2_Shape.zip downloaded and unzipped successfully.

Downloaded: WBD_02_HU2_Shape.zip
Extracted files to: /Users/alan/Data Science Projects/Unit-Hydrograph-Model/data/raw/json_docs/WBD_02_HU2_Shape
Deleted ZIP file: /Users/alan/Data Science Projects/Unit-Hydrograph-Model/data/raw/json_docs/WBD_02_HU2_Shape.zip
Function 'download_shp' executed in 1 minutes and 43.5544 seconds.

WBD_02_HU2_Shape.zip downloaded and unzipped successfully.

Downloaded: WBD_04_HU2_Shape.zip
Extracted files to: /Users/alan/Data Science Projects/Unit-Hydrograph-Model/data/raw/json_docs/WBD_04_HU2_Shape
Deleted ZIP file: /Users/alan/Data Science Projects/Unit-Hydrograph-Model/data/raw/json_docs/WBD

## 2. Digital Elevation Model (DEM)

DEM available dataset is available through TNM Rest API. Refer to [dataset documentation here](../../docs/markdown/datasets.md). It has 1-meter resolution and is used to automatically delineate watershed boundaries. 

### 2.1. Digital Elevation Model Dataset availability

Below are the available DEM for download.

In [11]:
# Load the bounding box for upper hudson basin
upper_hudson_basin_bbox_path = project_base_path / 'data' / 'raw'/ 'geo' /'json' / 'upper_hudson_basin_bbox.json'
with open(upper_hudson_basin_bbox_path, 'r') as f:
    upper_hudson_basin_bbox_dict = json.load(f)

# Build the box as a string for feeding the request in the parameters
corners = ['bottom_left', 'bottom_right', 'top_right', 'top_left']
pairs = [f"{upper_hudson_basin_bbox_dict[corner][0]} {upper_hudson_basin_bbox_dict[corner][1]}" for corner in corners]
bbox = ",".join(pairs)

# Define the base URL for the TNM API
base_url = "https://tnmaccess.nationalmap.gov/api/v1/"

product = "National Elevation Dataset (NED) 1/3 arc-second"

# Define parameters for the API request to query available datasets
params = {
    "polygon": bbox,  # Specify the area to search for
    "datasets": product,  
    "outputFormat": "JSON"  # Specify JSON output
}

# Send a GET request to the API
response_upper_hudson_basin = requests.get(base_url + "products", params=params)

# Check if the request was successful
if response_upper_hudson_basin.status_code == 200:
    # Parse the JSON response
    response_upper_hudson_basin_json = response_upper_hudson_basin.json()
    
    # Display the dataset information
    display(Markdown(f'**{len(response_upper_hudson_basin_json.get("items",[]))} files were found:**\n '))
    for dataset in response_upper_hudson_basin_json.get("items", []):
        print(f"- Name: {dataset['title']}")
        print(f"  Publication Date: {dataset['publicationDate']}")
        print(f"  Description: {dataset['body']}")
        print(f"  Metadata URL: {dataset['metaUrl']}\n")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response_upper_hudson_basin.status_code}")

**50 files were found:**
 

- Name: USGS 1/3 Arc Second n41w074 20211109
  Publication Date: 2021-11-09
  Description: This tile of the 3D Elevation Program (3DEP) seamless products is 1/3 Arc Second resolution. 3DEP data serve as the elevation layer of The National Map, and provide basic elevation information for Earth science studies and mapping applications in the United States. Scientists and resource managers use 3DEP data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. 3DEP data compose an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers consists of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. 3DEP data are updated continually as new data become available. Seamless 3DEP data are derived from diverse source data that are processed to a common coordinate system and unit of vertical measure. These

From the list above we observe that from the same area there is multiple `.tiff` files, from different dates. However, it is needed only the latest file. 

Below, we filter the list to have only the latest GeoTIFF for the each area and dowload those files.

In [12]:
def filter_latest_geotiff(data):

    region_latest = {}
    
    for item in data:
        # Extract region from title using regex pattern (e.g., n41w074)
        match = re.search(r'n\d+w\d+', item['title'], re.IGNORECASE)
        if not match:
            continue
        region = match.group(0).lower()

        # Parse publication date
        pub_date = datetime.strptime(item['publicationDate'], '%Y-%m-%d')

        # If region is new or found a later publication date, update the entry
        if region not in region_latest or pub_date > datetime.strptime(region_latest[region]['publicationDate'], '%Y-%m-%d'):
            region_latest[region] = item

    # Return unique latest items for each region
    return list(region_latest.values())

upper_hudson_dem_all_items = response_upper_hudson_basin_json.get("items",[])
upper_hudson_dem_filtered_data = filter_latest_geotiff(data = upper_hudson_dem_all_items)

# Display the dataset information
display(Markdown(f'**{len(upper_hudson_dem_filtered_data)} unique files were found:**\n '))
for dataset in upper_hudson_dem_filtered_data:
    print(f"- Name: {dataset['title']}")
    print(f"  Publication Date: {dataset['publicationDate']}")
    print(f"  Description: {dataset['body']}")
    print(f"  Metadata URL: {dataset['metaUrl']}\n")


**17 unique files were found:**
 

- Name: USGS 1/3 Arc Second n41w074 20240925
  Publication Date: 2024-09-25
  Description: This tile of the 3D Elevation Program (3DEP) seamless products is 1/3 Arc Second resolution. 3DEP data serve as the elevation layer of The National Map, and provide basic elevation information for Earth science studies and mapping applications in the United States. Scientists and resource managers use 3DEP data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. 3DEP data compose an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers consists of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. 3DEP data are updated continually as new data become available. Seamless 3DEP data are derived from diverse source data that are processed to a common coordinate system and unit of vertical measure. These

In [14]:
# Save the filtered response to a file
export_path_dem_ds = project_base_path / 'data' / 'json_docs' / 'dem_1_3_arcsec_dataset.json'

if not export_path_dem_ds.exists():
    try:
        with open(export_path_dem_ds, 'w') as f:
            json.dump(upper_hudson_dem_filtered_data, f, indent=4)
        print(f'\nDEM 1/3 arcsec Dataset saved successfully to {export_path_dem_ds}.\n')
    except Exception as err:
        print(f'Could not save the file. An error was encountered: {err}')
else:
    print(f'\nDEM 1/3 arcsec Dataset already exists at {export_path_dem_ds}.\n')


DEM 1/3 arcsec Dataset saved successfully to /Users/alan/Data Science Projects/Unit-Hydrograph-Model/data/json_docs/dem_1_3_arcsec_dataset.json.



### 2.2. Digital Elevation Model Download

In [16]:
# Read the json dataset document
export_path_dem_ds = project_base_path / 'data' / 'raw' / 'json_docs' / 'dem_1_3_arcsec_dataset.json'
with open(export_path_dem_ds, 'r') as file:
    dem_ds = json.load(file)

count = 0
total_files = len(dem_ds)

# Download the DEM 1/3 arcsec file
for item in dem_ds:
    count += 1
    try:
        url = item.get('downloadURL')
        
        item_base_name = os.path.basename(url)
        item_local_path = project_base_path / 'data' / 'geo' / 'raster' / 'dem13arcsec' / item_base_name
        
        if not item_local_path.exists():
            print(f'Downloading {item.get("title")} file {count} of {total_files}...')
            download_GeoTIFF(url=url, filename=item_local_path, chunk_size=1024*1024)
            
        else:
            print(f'{item_base_name} already exists.\n')

    except Exception as err:
        print(f'Failed to download file: {err}')
        continue

USGS_13_n41w074_20240925.tif already exists.

USGS_13_n41w075_20221115.tif already exists.

USGS_13_n41w076_20221115.tif already exists.

USGS_13_n42w073_20211109.tif already exists.

USGS_13_n42w074_20241010.tif already exists.

USGS_13_n42w075_20240925.tif already exists.

USGS_13_n42w076_20230227.tif already exists.

USGS_13_n43w073_20230117.tif already exists.

Downloaded: USGS_13_n43w074_20241010.tif0.00%)
Function 'download_GeoTIFF' executed in 12 minutes and 19.3284 seconds.

Downloaded: USGS_13_n43w075_20241010.tif0.00%)
Function 'download_GeoTIFF' executed in 10 minutes and 37.8858 seconds.

Downloaded: USGS_13_n43w076_20230227.tif0.00%)
Function 'download_GeoTIFF' executed in 7 minutes and 27.2915 seconds.

Downloaded: USGS_13_n44w074_20241010.tif0.00%)
Function 'download_GeoTIFF' executed in 7 minutes and 9.9678 seconds.

Downloaded: USGS_13_n44w075_20241010.tif0.00%)
Function 'download_GeoTIFF' executed in 5 minutes and 48.8984 seconds.

Downloaded: USGS_13_n44w076_20241010

## 3. National Land Cover Data

The National Land Cover data (Land Cover, Land Cover Confidence, Fraction Impervious Surface, Fraction Impervious Descripor) were donwloaded from MRLC download tool: given an area of interest it sends an e-mail with a link for the download. Because the file is too big (about 18Gb), a function has been written to allow it to resume download in case it halts for any reason. 

In [5]:
mrlc_data_urls = 'https://www.mrlc.gov/downloads/sciweb1/shared/mrlc/download-tool/NLCD_lUiv89Ym9tDA9GgEH1RN.zip'
mrlc_data_path = project_base_path / 'data' / 'raw' / 'geo' / 'raster' / 'NLCD' / 'NLCD.zip'

# Ensure folder and subfulders exist
mrlc_data_path.parent.mkdir(parents=True, exist_ok=True)

# Download the NLCD data
download_large_file(url=mrlc_data_urls, destination=mrlc_data_path, max_retries=5)

Starting download: /Users/alan/Data Science Projects/Unit-Hydrograph-Model/data/geo/raster/mrlc/NLCD.zip (0/19750974412 bytes)
Downloaded 19750974412/19750974412 bytes (100.00%)
Download completed: /Users/alan/Data Science Projects/Unit-Hydrograph-Model/data/geo/raster/mrlc/NLCD.zip
Function 'download_large_file' executed in 238 minutes and 48.6681 seconds.



## 4. Precipitation Data

15 minutes precipitation data are available in [NCDC NOAA website here](https://www1.ncdc.noaa.gov/pub/data/hpd/auto/v2/beta/15min/) for version 1 and [NOAA website here](https://gis.ncdc.noaa.gov/kml/precip_15.kmz) for version 2. First we download the data inventory to locate which stations fall down within the region of interest, second we download data for those precipitation stations. (Refere to [data preprocessing here](../preprocessing/data_prepocessing.ipynb), for the study region selection). Documentation may be accessed [here](https://www1.ncdc.noaa.gov/pub/data/hpd/auto/v2/beta/15min/readme.15min.txt) and [here](../../docs/other/readme.15min.txt).

### 4.1. Precipitation Station Inventory

There are two versions of precipitation data available from NOAA. Version 1 has data up to December 2013. Version two has data starting the same period as version 1 to the present but, there are less precipitatin stations available as compared to version 1. In this study is used both dataset. Dataset are available for direct download on the above mentioned links. The inventory contain the following relevant attributes:
- Version 1:
    - Name: the name of station.
    - StnID: station identification code.
    - elev: elevation.
- Version 2:
    -  StnID: station identification code.
    -  Lat: latitude.
    -  Lon:lontitude.
    -  Elev: elevation of the station.
    -  Name: the name of the station. 
    -  Sample_Interval (min): is in units of minute and indicates the typical time between sampling.
    -  UTC_Offset: is the number of hours the station's local time is offset from GMT.
    -  POR_Date_Range: first and last year-month-day of the station's Period of Record.
    -  PCT_Last_Half_Good: is the percentage of non-missing and non-flagged observations during the last half of the station's POR.  

In [8]:
# Read precipitation data inventory from website
ppt_stations_inventory = pd.read_csv(filepath_or_buffer='https://www1.ncdc.noaa.gov/pub/data/hpd/auto/v2/beta/15min/hpd-stations-inventory.15min.csv')

In [13]:
# Save the precipitation inventory data
ppt_stations_inventory_path = project_base_path / 'data' / 'geo' / 'json' / 'ppt_stations_inventory.json'

# Save the file in csv format
if not ppt_stations_inventory_path.exists():
    try:
        ppt_stations_inventory_path.parent.mkdir(parents=True, exist_ok=True)
        ppt_stations_inventory.to_json(ppt_stations_inventory_path)
        print('Successfully saved precipitation inventory data.')
    except Exception as err:
        print(f'Failed to save precipitation inventory data: {err}')
else:
    print('File already exists.')

File already exists.


### 4.2. Precipitation Historical Data

Historical data for precipitation dataset version 1 is available through NOAA API (see the docs) and through FTP for version 2.

First, we load the metadata information about the the precipitation stations to get the ID to download data for. We Download both dataset for starting at year 1985 up to present, acording to data availability.

#### 4.2.2. Download Historical Precipitation Data Version 2

In [3]:
# Load metada for historical precipitation file version 2
precip_v2_metada_path = project_base_path / 'data/clean/json/precip_stations_v2_codes.json'
with open(precip_v2_metada_path, 'r') as file:
    precip_v2_metada = json.load(file)
precip_v2_metada

NameError: name 'project_base_path' is not defined

In [4]:
USC00305435 = pd.read_csv('https://www1.ncdc.noaa.gov/pub/data/hpd/auto/v2/beta/15min/all_csv/USC00305435.15m.csv')

/var/folders/6x/tz4_svyx1mv9mc68z3jl0_s00000gn/T/ipykernel_1318/935787855.py:1: DtypeWarning: Columns (9,14,19,24,29,34,39,44,49,54,59,64,69,74,79,84,89,94,99,104,109,114,119,124,129,134,139,144,149,154,159,164,169,174,179,184,189,194,199,204,209,214,219,224,229,234,239,244,249,254,259,264,269,274,279,284,289,294,299,304,309,314,319,324,329,334,339,344,349,354,359,364,369,374,379,384,389,394,399,404,409,414,419,424,429,434,439,444,449,454,459,464,469,474,479,484) have mixed types. Specify dtype option on import or set low_memory=False.
  USC00305435 = pd.read_csv('https://www1.ncdc.noaa.gov/pub/data/hpd/auto/v2/beta/15min/all_csv/USC00305435.15m.csv')


In [18]:
USC00305435

,STATION,LATITUDE,LONGITUDE,ELEVATION,DATE,ELEMENT,0000Val,0000MF,0000QF,0000S1,...,2345Val,2345MF,2345QF,2345S1,2345S2,DlySum,DlySumMF,DlySumQF,DlySumS1,DlySumS2
0,USC00305435,41.6667,-74.7833,338.9,1984-01-10,QPCP,-9999,,D,6,...,1,,,6,,16,,P,,C
1,USC00305435,41.6667,-74.7833,338.9,1984-01-11,QPCP,0,Z,,6,...,0,Z,,6,,3,,,,C
2,USC00305435,41.6667,-74.7833,338.9,1984-01-12,QPCP,0,Z,,6,...,0,Z,,6,,0,,,,C
3,USC00305435,41.6667,-74.7833,338.9,1984-01-13,QPCP,0,Z,,6,...,0,Z,,6,,0,,,,C
4,USC00305435,41.6667,-74.7833,338.9,1984-01-14,QPCP,0,Z,,6,...,1,,,6,,3,,,,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9830,USC00305435,41.6667,-74.7833,338.9,2023-10-28,QPCP,0,,,H,...,0,,,H,R,0,,,,C
9831,USC00305435,41.6667,-74.7833,338.9,2023-10-29,QPCP,0,,,H,...,0,,,H,R,72,,,,C
9832,USC00305435,41.6667,-74.7833,338.9,2023-10-30,QPCP,0,,,H,...,0,,,H,R,57,,,,C
9833,USC00305435,41.6667,-74.7833,338.9,2023-10-31,QPCP,0,,,H,...,0,,,H,R,1,,,,C


In [ ]:
# Download data through FTP server

USC00305435.to_parquet('test.parquet')

In [18]:
USC00305435.tail()

,STATION,LATITUDE,LONGITUDE,ELEVATION,DATE,ELEMENT,0000Val,0000MF,0000QF,0000S1,...,2345Val,2345MF,2345QF,2345S1,2345S2,DlySum,DlySumMF,DlySumQF,DlySumS1,DlySumS2
9830,USC00305435,41.6667,-74.7833,338.9,2023-10-28,QPCP,0,,,H,...,0,,,H,R,0,,,,C
9831,USC00305435,41.6667,-74.7833,338.9,2023-10-29,QPCP,0,,,H,...,0,,,H,R,72,,,,C
9832,USC00305435,41.6667,-74.7833,338.9,2023-10-30,QPCP,0,,,H,...,0,,,H,R,57,,,,C
9833,USC00305435,41.6667,-74.7833,338.9,2023-10-31,QPCP,0,,,H,...,0,,,H,R,1,,,,C
9834,USC00305435,41.6667,-74.7833,338.9,2023-11-01,QPCP,0,,,H,...,-9999,,,H,R,0,,P,,C


In [ ]:
parameters2 = {'stationid':'COOP:267369',
               'datasetid':'PRECIP_15',
               'startdate':'2012-10-31',
               'enddate':'2012-12-31'}
response2 = requests.get(url=base_url + 'data', headers=headers, params=parameters2)
response2.json()